In [2]:
import cv2
import numpy as np
import argparse
import time
from collections import OrderedDict
from scipy.spatial import distance as dist

In [3]:
clusters = []

In [4]:
modelConfiguration = "D:\Repository\darknet\cfg\yolov3.cfg"
modelWeights = "D:\Repository\darknet\yolov3.weights"
cocoFile = "D:\Repository\darknet\cfg\coco.names"

In [5]:
def load_yolo():
    net = cv2.dnn.readNet(modelWeights, modelConfiguration)
    classes = []
    with open(cocoFile, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
        
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    np.random.seed(1)
    colors = np.random.uniform(0, 255, size = (len(classes), 3))
    return net, classes, colors, output_layers


def detect_objects(img, net, output_layers):
    blob = cv2.dnn.blobFromImage(img, scalefactor = 1/255, size = (384, 384), mean = [0, 0, 0], swapRB = True, crop = False)
    net.setInput(blob)
    outputs = net.forward(output_layers)
    return blob, outputs


def get_box_dimension(outputs, height, width):
    global clusters
    boxes = []
    confs = []
    class_ids = []
    for output in outputs:
        for detect in output:
            scores = detect[5:]
            class_id = np.argmax(scores)
            conf = scores[class_id]
            if conf > 0.5:
                center_x = int(detect[0] * width)
                center_y = int(detect[1] * height)
                w = int(detect[2] * width)
                h = int(detect[3] * height)
                x = int(center_x - w/2)
                y = int(center_y - h/2)
                
                class_ids.append(class_id)
                boxes.append([x, y, w, h])
                confs.append(float(conf))
                
    return boxes, confs, class_ids


def draw_labels(boxes, confs, colors, class_ids, classes, img):
    cv2.line(img, (0, 500), (1280, 500), (0, 255, 0), 1)
    indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
#     print('indexes', indexes)
#     print('boxes', boxes)
    font = cv2.FONT_HERSHEY_PLAIN
    
#     for idx, i in enumerate(boxes):
# #         color = colors[i]
#         label = 'index : {}'.format(str(idx))
#         cv2.rectangle(img, (i[0], i[1]), (i[0] + i[2], i[1] + i[3]), (255, 0 ,0), 2)
#         cv2.putText(img, label, (i[0], i[1] - 5), font, 1, (255, 0 ,0), 1)
        
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = '{} index : {}'.format(str(classes[class_ids[i]]), str(i))
            color = (255, 0 ,0)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
    cv2.imshow('Image', img)

In [6]:
class CentroidTracker():
    def __init__(self, maxDisappeared = 50):
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.maxDisappeared = maxDisappeared
        
    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1
    
    def deregister(self, objectId):
        del self.objects[objectID]
        del self.disappeared[objectID]
        
    def update(self, rects):
        if len(rects) == 0:
            for objectID in list(self.disappeared.keys()):
                print(self.disappeared.keys(), '\n')
                self.disappeared[objectID] += 1
                
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects
    
        inputCentroids = np.zeros((len(rects), 2), dtype = 'int')
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)
            
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())
            
            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]
            
            usedRows = set()
            usedCols = set()
            
            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0
                
                usedRows.add(row)
                usedCols.add(col)
                
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[0])).difference(usedCols)
                
            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1
                    
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
                        
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])
        return self.objects


In [9]:
confidence = 0.5
ct = CentroidTracker()

net, classes, colors, output_layers = load_yolo()
video_path = 'traffic.mp4'

cap = cv2.VideoCapture(video_path)

while True:
    _, frame = cap.read()
    
    (H, W) = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(frame, scalefactor = 1/255, size = (220, 220), mean = [0, 0, 0])
    net.setInput(blob)
    outputs = net.forward(output_layers)
    
    rects = []
    boxes = []
    confs = []
    
    for output in outputs:
        for detect in output:
            scores = detect[5:]
            class_id = np.argmax(scores)
            conf = scores[class_id]
            if conf > confidence:
                box = detect[:4] * np.array([W, H, W, H])
                boxes.append(box.astype("int"))
                confs.append(float(conf))
                
    indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
    
    for i in range(len(boxes)):
        if i in indexes:
            rects.append(boxes[i])
            (startX, startY, endX, endY) = boxes[i].astype("int")
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            
    
                
#                 (startX, startY, endX, endY) = box.astype("int")
#                 cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
        
#     for i in range(0, outputs.shape[0]):
#         scores = detect[5:]
#         class_id = np.argmax(scores)
#         conf = scores[class_id]
#         if conf > confidence:
#             box = outputs[0, 0, i, 3:7] * np.array([W, H, W, H])
#             rects.append(box.astype("int"))
            
#             (startX, startY, endX, endY) = box.astype("int")
            
#             cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
#     print(rects, '\n')
    objects = ct.update(rects)
    
    for (objectID, centroid) in objects.items():
        text = "ID {}".format(objectID)
        cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
        
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
        
cv2.destroyAllWindows()

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 

odict_keys([0, 1, 2, 3]) 



In [10]:
def start_video(video_path):
    model, classes, colors, output_layers = load_yolo()
    confidence = 0.5
    ct = CentroidTracker()
    cap = cv2.VideoCapture(video_path)
    while True:
        _, frame = cap.read()
        height, width, channels = frame.shape
        blob, outputs = detect_objects(frame, model, output_layers)
        boxes, confs, class_ids = get_box_dimension(outputs, height, width)
        
#         indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
#         rects = []
#         for i in range(len(boxes)):
#             if i in indexes:
#                 rects.append(boxes[i])
#                 (startX, startY, endX, endY) = boxes[i]
#                 cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
        
#         objects = ct.update(rects)

#         for (objectID, centroid) in objects.items():
#             text = "ID {}".format(objectID)
#             cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
#             cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
        
        draw_labels(boxes, confs, colors, class_ids, classes, frame)
#         print('=====================')
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    

start_video('traffic.mp4')